In [ ]:
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
data_dir = '/content/drive/MyDrive/CS/CS691-692/data' 
input_shape = (256, 256, 3)
lr = 1e-5
epochs = 50
batch_size = 64

In [ ]:
# Remove dodgy images
import cv2
import imghdr

image_exts = ['jpeg', 'jpg', 'bmp', 'png']


In [ ]:
# making sure image exists in image directory
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

In [ ]:
# Load data
data = tf.keras.utils.image_dataset_from_directory(data_dir, image_size=input_shape[:2], batch_size=batch_size)
data = data.map(lambda x, y: (x/255, y))

In [ ]:
# Split data
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [ ]:
# Build ResNet50 model
base_model = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)

In [ ]:
# Add a new classification layer on top
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [ ]:
# Create a new model
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Freeze the base ResNet50 layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=5, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=1e-7)

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train model
history = model.fit(train, 
                    epochs=50,
                    validation_data=val,
                    callbacks=[learning_rate_reduction])

In [ ]:
# Plot accuracy and loss
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

In [ ]:
# Evaluate model
model.evaluate(test)


5/5 [==============================] - 10s 227ms/step - loss: 0.4921 - accuracy: 0.7719


[0.4920650124549866, 0.7718750238418579]

#Test

In [ ]:
import cv2

In [ ]:
img = cv2.imread('/content/drive/MyDrive/CS/CS691-692/Melanoma.jpg')
plt.imshow(img)
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
test = plt.imshow(resize.numpy().astype(int))

In [ ]:
import numpy as np
#y prediction
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
yhat

#Save the model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('/content/drive/MyDrive/CS/CS691-692/models','imageclassifier-resnet50.h5'))